In [1]:
import pdfminer
import json

read_pdf = None
# f = open("Homework_1.pdf", "rb")
# read_pdf = pdfminer.
print(pdfminer)

<module 'pdfminer' from 'c:\\Users\\chp20\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\pdfminer\\__init__.py'>


In [5]:
#%pip uninstall pdfminer.six
#%pip install pdfminer.six
import pdfminer.high_level as pdf_hl
output = pdf_hl.extract_pages("Homework_1.pdf")
#print(output)
output = list(output)
print(output)

[<LTPage(1) 0.000,0.000,612.000,792.000 rotate=0>, <LTPage(2) 0.000,0.000,612.000,792.000 rotate=0>]


In [6]:
import PIL
from PIL import Image
from PIL import ImageDraw
#images = []

In [9]:
#output = list(output)
images = []
for page in output:
    print(page)
    p_bbox = page.bbox
    width = p_bbox[2] - p_bbox[0]
    height = p_bbox[3] - p_bbox[1]
    p_image = PIL.Image.new('RGB', (width, height), 'white')
    p_draw = ImageDraw.Draw(p_image)
    print("-----")
    for line in page:
        print(line)
        bbox = line.bbox
        print(bbox)
        width_line = bbox[2] - bbox[0]
        height_line = bbox[3] - bbox[1]
        x0 = bbox[0] - p_bbox[0]
        x1 = bbox[2] - p_bbox[0]
        #y coordinates are flipped
        y0 = height - (bbox[3] - p_bbox[1])
        y1 = height - (bbox[1] - p_bbox[1])
        p_draw.rectangle([x0, y0, x1, y1], outline='red')
        #p_draw.text((bbox[0] - p_bbox[0], bbox[1] - p_bbox[1]), line.get_text(), fill='black')
    images.append(p_image)
    #p_image.show()
    #break

for ind, img in enumerate(images):
    img.save(f"page_{ind}.png", "PNG")

<LTPage(1) 0.000,0.000,612.000,792.000 rotate=0>
-----
<LTTextBoxHorizontal(0) 77.978,678.246,181.068,719.094 'MATH 420\nHomework 1\nSeptember 20, 2023\n'>
(77.978, 678.246304, 181.0676896, 719.093504)
<LTTextBoxHorizontal(1) 394.289,707.138,428.911,719.094 'Name:\n'>
(394.2886816, 707.1383040000001, 428.91094080000005, 719.093504)
<LTTextBoxHorizontal(2) 215.679,636.222,396.322,648.178 'Submit your work on separate paper.\n'>
(215.679, 636.2224384, 396.322072, 648.1776384000001)
<LTTextBoxHorizontal(3) 87.223,608.825,452.965,621.247 '1. Find a certificate of infeasibility for the system Ax = b, x ≥ 0 given by\n'>
(87.22300000000001, 608.8254383999999, 452.9648864, 621.2468911999999)
<LTTextBoxHorizontal(4) 240.969,588.013,248.940,599.968 '\uf8ee\n'>
(240.96900000000002, 588.0128799999999, 248.93953184000003, 599.9680799999999)
<LTTextBoxHorizontal(5) 240.969,566.494,248.940,578.449 '\uf8f0\n'>
(240.96900000000002, 566.4938799999999, 248.93953184000003, 578.4490799999999)
<LTTextBoxHor